PDF RAG 시스템 구축 코드

In [2]:
!pip install chromadb sentence-transformers pypdf # 벡터DB (chromadb), 임베딩 모델 (sentence-transformers), 그리고 PDF 파일을 읽는 데 특화된 라이브러리 (pypdf)를 한 번에 설치

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 85.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 82.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.4/132.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 18.4 MB/s eta

In [6]:
import os, re, uuid    # uuid : 고유 식별 id 생성용
from typing import List   # type hint 기능 제공 (가독성)
from sentence_transformers import SentenceTransformer   # 문장 단위의 의미 임베딩 라이브러리
from chromadb import PersistentClient
import pypdf

PDF_PATH = "sample.pdf"
CHROMA_DIR = ".chroma_pdf_demo" # DB 데이터가 저장될 폴더
COLLECTION = "pdf_docs"         # DB 내의 데이터 테이블
MODEL_NAME = "all-MiniLM-L6-v2" # 벡터로 바꾸는 기계


In [7]:
def read_pdfFunc(path:str) -> str:
  if not os.path.exists(path):  # # 1. 파일 경로 확인
    raise FileNotFoundError(f"허걱 pdf 파일이 없어요:{path}")

  text_pages = []  # 페이지별 텍스트 저장 리스트
  try:
    with open(path, 'rb') as f:      # ⭐'rb'는 파일을 '바이너리 읽기' 모드로 열기.
      reader = pypdf.PdfReader(f)   # pypdf 리더 객체 생성
      for i, page in enumerate(reader.pages):   # PDF 내 모든 페이지를 순회
        txt = page.extract_text() or ""         # 각 페이지에서 텍스트를 추출
        text_pages.append(txt)
    return "\n\n".join(text_pages)              # 추출된 모든 페이지 텍스트를 두 줄의 개행으로 연결해 반환
  except Exception:
    raise RuntimeError(f"pdf 추출 실패")

# 문단 단위로 분리 (Chunking)
def split_paragraphFunc(text:str, min_len:int=40) -> List[str]:
  chunks = re.split(r"\n\s*\n+", text)   # 빈 줄 기준 문단 분리
  chunks = [re.sub(r"\s+", " ", p).strip() for p in chunks] # 공백을 하나로 통일하고 앞뒤 공백 제거
  return [p for p in chunks if len(p) >= min_len ]# 최소 길이(40자)보다 긴 청크만 반환

# 임베딩 모델 로드 (매번 호출되지만, 메모리에 로드되어 있으므로 빠름)
def embedderFunc(name:str=MODEL_NAME):
  return SentenceTransformer(name)

# 텍스트를 벡터로 변환 (Embedding)
def embedFunc(model, texts:List[str])-> List[List[float]]:
  return model.encode(texts, normalize_embeddings=True).tolist()  # normalize_embeddings=True: 벡터의 크기를 1로 정규화. (⭐ 매우 중요!)

# ChromaDB 컬렉션 생성/가져오기
def get_collectionFunc(chroma_dir:str, name:str):
  client = PersistentClient(path=chroma_dir)
  return client.get_or_create_collection(name)

# pdf 파일을 읽어 VectorDb에 저장 (Upsert: 저장 또는 업데이트)
def upsert_pdfFunc(pdf_path:str):
  full_text = read_pdfFunc(pdf_path)  # PDF에서 텍스트 추출
  # print(full_text)

  if not full_text.strip():
    print('pdf에서 추출된 자료가 없음')
    return 0

  chunks = split_paragraphFunc(full_text, min_len=40) # 텍스트를 문단 청크로 분리
  # print(len(chunks))   # 2
  if not chunks:
    print('저장할 문단이 없어요')
    return 0

  # 임베딩 및 DB 정보 준비
  model = embedderFunc(MODEL_NAME)
  embs = embedFunc(model, chunks) # 문단 청크를 벡터로 변환
  # print(embs)

  metas = []  # 메타데이터 (꼬리표) 리스트 준비
  for c in chunks:
    metas.append(
        {
            "source":os.path.basename(pdf_path),
            "len":len(c)
            # 파일명과 청크 길이 저장
        }
    )

  collection = get_collectionFunc(CHROMA_DIR, COLLECTION)
  ids = [str(uuid.uuid4()) for _ in chunks] # 각 청크에 고유 ID 할당
  collection.add(ids=ids, documents=chunks, embeddings=embs, metadatas=metas) # ChromaDB에 최종 저장
  return len(chunks)


def searchFunc(query:str, k:int):
  # 1. 쿼리 임베딩: 질문도 DB에 저장된 벡터와 비교하기 위해 벡터로 변환
  model = embedderFunc(MODEL_NAME)
  q_emb = embedFunc(model, [query])
  # 2. DB 연결 및 쿼리 요청
  collection = get_collectionFunc(CHROMA_DIR, COLLECTION)
  res = collection.query(query_embeddings=q_emb, n_results=k) # 쿼리 벡터와 가장 가까운 k개의 결과를 찾아달라고 요청
  # 3. 결과 추출
  docs = res.get('documents', [[]])[0]      # 예외 방지용 패턴 실제 텍스트 내용 추출
  metas = res.get('metadatas', [[]])[0]    # 메타데이터 추출
  ids = res.get('ids', [[]])[0]
  dists = res.get('distances', [[]])[0]    # 쿼리 벡터와의 거리 추출
  # 4. 결과 출력
  for i, (doc, meta, _id, dist) in enumerate(zip(docs, metas, ids, dists)):
    print(f'\n[{i}] id={_id}')
    print(f'source={meta.get('source')}, len={meta.get('len')}, distance={dist:.4f}') # dist 값이 작을수록 질문과 의미적으로 유사하다는 뜻
    print(doc[:300] + ("..." if len(doc) > 300 else ""))

꼭 기억해야 할 핵심 내용

'rb' (Read Binary) : PDF 파일은 텍스트 파일과 달리 이진(Binary) 형식으로 저장되므로, 파이썬에서 반드시 'rb' 모드로 열어야 pypdf가 내용을 인식할 수 있음.

reader.pages : pypdf가 PDF 파일의 페이지 목록을 반환. PDF는 페이지 개념이 있으므로, 각 페이지를 순회하며 텍스트를 추출해야 함.

page.extract_text() : pypdf의 핵심 기능. PDF의 복잡한 레이아웃을 무시하고 실제 텍스트 내용만 추출 .

Chunking의 목표: RAG에서 검색 단위를 결정하는 단계. 너무 길면 LLM이 문맥을 놓치고, 너무 짧으면 의미가 불완전

공백 정규화 (re.sub(r"\s+", " ", p)) : PDF에서 텍스트를 추출하면 불필요한 공백이나 개행 문자가 많아지는데, 이 정규식은 모든 공백(탭, 개행 포함)을 하나의 띄어쓰기로 깔끔하게 정리

임베딩 정규화 : normalize_embeddings=True는 필수. 벡터의 크기를 1로 맞춰서 유사도 검색(코사인 유사도) 시 속도와 정확성을 높임.

RAG 파이프라인 : upsert_pdfFunc가 PDF 읽기 → 청크 분리 → 벡터화 → DB 저장이라는 RAG 지식 주입 파이프라인 전체를 책임


메타데이터 사용 : source와 len을 저장하는 것은 검색 결과가 나온 후 "이 정보가 어떤 파일의 몇 번째 줄에서 왔는지를 LLM이나 사용자에게 알려줄 수 있는 근거 정보



질문과 지식의 동등성 : 질문(query)과 DB의 지식 청크는 모두 같은 임베딩 모델을 통해 벡터화. 같은 공간에서 의미적 거리를 정확하게 측정할 수 있음.

n_results=k : 질문과 가장 의미가 유사한 상위 k개의 텍스트 조각을 가져옴. 이 조각들이 LLM에게 전달될 문맥(Context)이 됨

distance 값 : 값이 작을수록 질문의 의미와 가장 근접한 지식이라는 의미

In [10]:
if __name__ == "__main__":
  n = upsert_pdfFunc(PDF_PATH)

  print(f"\n저장된 문단 수 : {n}")
  searchFunc("강남에 지역 문화적 뉘앙스를 더하면 로컬 개발자들의 개성이 살아날 수 있다",  k=3)


저장된 문단 수 : 10

[0] id=72b66417-58f4-4282-bd2f-31839efaec27
source=sample.pdf, len=1101, distance=0.6822
이 맺히었다 김첨지의 눈시울도 뜨끈뜨끈하였다. . 이 환자가 그러고도 먹는 데는 물리지 않았다 사흘 전부터 설렁탕 국물이. 마시고 싶다고 남편을 졸랐다. 이런 오라질 년 조밥도 못 먹는 년이 설렁탕은 또 처먹고 지랄병을 하“ ! . 게.” 라고 야단을 쳐보았건만 못 사주는 마음이 시원치는 않았다, , . 인제 설렁탕을 사줄 수도 있다 앓는 어미 곁에서 배고파 보채는 개똥이. 세살먹이 에게 죽을 사줄 수도 있다 팔십 전을 손에 쥔 김 첨지의 마음( ) - 은 푼푼하였다. 그러나 그의 행운은 그걸로 그치지 않았다 땀과 빗물이 섞여 흐르...

[1] id=e496de06-307f-4172-81fe-202bb331d1cd
source=sample.pdf, len=1094, distance=0.7253
나가지 말라도 그래 그러면 일찍이 들어와요“ , .” 하고 목메인 소리가 뒤를 따랐다, . 정거장까지 가잔 말을 들은 순간에 경련적으로 떠는 손 유달리 큼직한 눈 울 듯한 아내의 얼굴이 김첨지의 눈앞에 어른어른하였다. 그래 남대문 정거장까지 얼마란 말이요“ ?” 하고 학생은 초조한 듯이 인력거꾼의 얼굴을 바라보며 혼자말같이, 인천 차가 열한 점에 있고 그 다음에는 새로 두 점이든가“ .” 라고 중얼거린다. 일 원 오십 전만 줍시요“ .” 이 말이 저도 모를 사이에 불쑥 김첨지의 입에서 떨어졌다 제 입으로 부. 르고도 스스로 그 엄청...

[2] id=25674980-4a8b-435a-9c68-9ac47e30fc49
source=sample.pdf, len=1266, distance=0.7308
니 김첨지는 인력거를 쥔 채 길 한복판에 엉거주춤 멈춰 있지 않은가. 예 예“ , .” 하고 김첨지는 또다시 달음질하였다 집이 차차 멀어 갈수록 김첨지의 걸, . 음에는 다시금 신이 나기 시작

PDF 기반 RAG 시스템 구축: 꼭 기억해야 할 핵심 5가지
1. PDF 특화된 전처리 (가장 큰 변화)

    필요성: PDF 파일은 단순히 open() 함수로 읽을 수 없고, 페이지 구조가 섞여 있어 텍스트를 추출하는 특수 도구가 필요.

    핵심 도구: pypdf 라이브러리.

    원리: read_pdfFunc 함수에서 파일을 바이너리 모드('rb')로 열고, pypdf.PdfReader를 사용하여 페이지별로 텍스트를 추출한 후 , 모든 페이지 텍스트를 합쳐서 하나의 긴 문자열로 만듦.
    실무 포인트: PDF를 다룰 때는 항상 pypdf나 PyMuPDF와 같은 전용 라이브러리를 사용하고, 파일을 'rb' 모드로 여는 것.

2. 강화된 청크 분리 (Chunking)
    함수: split_paragraphFunc(text, min_len=40)

    원리: 이전 텍스트 파일에서는 빈 줄(\n\s*\n+)로 문단을 나누는 것 외에, PDF 추출 과정에서 발생할 수 있는 지저분한 공백을 제거(re.sub(r"\s+", " ", p)).

    최소 길이: min_len=40으로 설정하여, 정보량이 부족한 너무 짧은 청크를 제거함으로써 검색 품질(Relevance)을 유지

    실무 포인트: re.sub(r"\s+", " ", p)는 PDF처럼 구조화가 복잡한 문서에서 텍스트를 추출할 때 데이터 정제를 위해 매우 유용한 정규 표현식 패턴.

3. 임베딩의 불변의 원칙 (정규화)
    함수: embedFunc(model, texts)

    핵심 코드: model.encode(texts, normalize_embeddings=True)

    원리: 질문(Query)이든, DB에 저장될 지식 청크(Chunk)든, 모든 벡터는 반드시 동일한 방식으로 생성되어야 함. 특히 normalize_embeddings=True를 통해 모든 벡터의 크기를 1로 맞춰서 코사인 유사도 계산을 최적화하고 정확성을 높임.

    ⭐꼭 기억: RAG에서 임베딩 모델과 정규화는 지식의 언어를 통일하는 가장 기본적인 규칙.

4. RAG 파이프라인: upsert와 search
    upsert_pdfFunc (지식 주입): 이 함수가 RAG의 지식 기반 구축 단계. PDF 텍스트를 가져와서 → 청크로 분리하고 → 벡터화(임베딩)하여 → ChromaDB의 특정 컬렉션(pdf_docs)에 ID, 텍스트, 벡터, 메타데이터를 묶어 한 번에 저장.

    searchFunc (지식 검색): 이 함수가 RAG의 검색 단계. 사용자의 질문을 벡터화하여 (q_emb) → DB에 질의(collection.query)합니다. 이때 distance 값이 작을수록 질문과 의미가 가장 유사한 지식 청크가 됨.

    파이프라인 이해: 지식 주입(upsert)과 지식 검색(search)의 흐름이 곧 RAG 시스템의 전체 동작 과정.

5. 메타데이터의 중요성 (증거 확보)

    저장 정보: metas 리스트에 {"source": pdf_path, "len": len(c)} 등의 부가 정보를 저장.

    역할: searchFunc의 결과 출력에서 source와 distance를 함께 보여주는 것은 LLM에게 답변을 받은 후 "이 답변의 근거는 어느 PDF 파일의 몇 번째 문단에서 왔는가"를 확인할 수 있게 해줌.

    실무 포인트: 메타데이터는 RAG 답변의 투명성(Transparency)과 신뢰성(Trustworthiness)을 확보하는 데 필수적.